# 财务舞弊识别实验1 - 数据预处理步骤指南

本Notebook基于 `preprocess_data_balanced.py` 脚本，将数据预处理流程拆分为多个步骤，方便逐步执行和调试。

## 📋 流程概览

1. **Step 0**: 环境准备与导入
2. **Step 1**: 数据理解与探索
3. **Step 2**: 读取和规范化单个表
4. **Step 3**: 数据集成（横向并表）
5. **Step 4**: 生成违规标签
6. **Step 5**: 数据清洗
7. **Step 6**: 生成ST标签
8. **Step 7**: 准备最终输出
9. **Step 8**: 质量校验
10. **Step 9**: 导出文件

---

## 🎯 使用说明

- 按顺序执行每个步骤的代码单元格
- 每个步骤都有检查点，确保数据质量
- 可以随时查看中间结果
- 遇到问题可以单独调试某个步骤


## Step 0: 环境准备与导入

导入必要的库，设置工作目录和参数。


In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import os
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# 设置参数
DATA_DIR = 'Dataset'
OUTPUT_DIR = 'Insight_output'
GROUP_ID = '1'

# 确保输出目录存在
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 数据文件路径配置
data_files = {
    'solvency': os.path.join(DATA_DIR, '偿债能力', 'FI_T1.xlsx'),
    'disclosure': os.path.join(DATA_DIR, '披露财务指标', 'FI_T2.xlsx'),
    'operation': os.path.join(DATA_DIR, '经营能力', 'FI_T4.xlsx'),
    'profit': os.path.join(DATA_DIR, '盈利能力', 'FI_T5.xlsx'),
    'risk': os.path.join(DATA_DIR, '风险水平', 'FI_T7.xlsx'),
    'growth': os.path.join(DATA_DIR, '发展能力', 'FI_T8.xlsx'),
    'pershare': os.path.join(DATA_DIR, '每股指标', 'FI_T9.xlsx'),
    'dividend': os.path.join(DATA_DIR, '股利分配', 'FI_T11.xlsx'),
    'violation': os.path.join(DATA_DIR, '违规信息总表', 'STK_Violation_Main.xlsx')
}

# Typrep 优先级：K(合并) > C(合并调整) > A(年报) > B(半年报) > 其他
typrep_priority = {'K': 1, 'C': 2, 'A': 3, 'B': 4, 'S': 5, 'H': 6, 'F': 7, 'E': 8, 'N': 9}

# 最终列顺序
column_order = [
    'Stkcd', 'Accper', 'Typrep', 'Indcd', 'isviolation', 'isST',
    'F040101B', 'F040202B', 'F040203B', 'F040205C', 'F040401B', 'F040503B', 'F040505C',
    'F040803B', 'F040805C', 'F041203B', 'F041205C', 'F041301B', 'F041403B', 'F041405C',
    'F041703B', 'F041705C', 'F041803B', 'F041805C',
    'F050104C', 'F050204C', 'F053201B', 'F053301C', 'F052401B', 'F053202B',
    'F080102A', 'F081002B', 'F082601B', 'F080603A',
    'F070101B', 'F070201B', 'F070301B',
    'F090102B', 'F020108',
    'F110101B', 'F110301B', 'F110801B'
]

print("=" * 80)
print("财务舞弊识别实验1 - 数据预处理（平衡版）")
print("策略：三键 (Stkcd, Year, Typrep) + 智能去重避免数据爆炸")
print("=" * 80)
print(f"\n开始时间: {datetime.now()}")
print(f"数据目录: {DATA_DIR}")
print(f"输出目录: {OUTPUT_DIR}")
print(f"组号: {GROUP_ID}")


## Step 1: 数据理解与探索

检查数据文件是否存在，查看数据的基本信息。


In [ ]:
# 检查数据文件是否存在
print("检查数据文件...")
for name, filepath in data_files.items():
    exists = os.path.exists(filepath)
    status = "✓" if exists else "✗"
    print(f"{status} {name:15s}: {os.path.basename(filepath)}")

# 读取一个示例文件查看结构（经营能力表）
if os.path.exists(data_files['operation']):
    print("\n" + "=" * 80)
    print("示例：经营能力表 (FI_T4.xlsx) 前5行")
    print("=" * 80)
    df_sample = pd.read_excel(data_files['operation'], nrows=5)
    print(f"列数: {len(df_sample.columns)}")
    print(f"列名: {list(df_sample.columns)}")
    print("\n数据预览:")
    print(df_sample)


## Step 2: 辅助函数定义

定义数据预处理所需的辅助函数。


In [ ]:
def read_excel_safe(filepath):
    """安全读取Excel文件"""
    try:
        print(f"正在读取: {os.path.basename(filepath)}")
        df = pd.read_excel(filepath)
        print(f"  成功读取 {len(df)} 行")
        return df
    except Exception as e:
        print(f"  读取失败: {str(e)}")
        return None

def standardize_stock_code(code):
    """标准化股票代码为6位字符串"""
    if pd.isna(code):
        return None
    try:
        return str(int(code)).zfill(6)
    except:
        return str(code).zfill(6)

def extract_year(date_str):
    """从日期字符串提取年份"""
    if pd.isna(date_str):
        return None
    try:
        if isinstance(date_str, str):
            return int(date_str.split('-')[0])
        elif isinstance(date_str, (pd.Timestamp, datetime)):
            return date_str.year
        else:
            return int(date_str)
    except:
        return None

print("辅助函数定义完成 ✓")


## Step 3: 读取和规范化单个表

逐个读取财务表，进行规范化处理：
- 处理中文列名（偿债能力表）
- 标准化键字段（Stkcd, Accper, Typrep）
- 智能去重（按Typrep优先级）


In [ ]:
def load_and_prepare_table(name, filepath):
    """读取、规范化并智能去重单个主题表"""
    if not os.path.exists(filepath):
        print(f"⚠️ {name} 文件不存在")
        return None
    
    df = read_excel_safe(filepath)
    if df is None:
        return None
    
    # 处理偿债能力表的中文列名
    if name == 'solvency':
        df.columns = [col.strip().strip("'") for col in df.columns]
        column_mapping = {}
        for col in df.columns:
            if '股票代码' in col:
                column_mapping[col] = 'Stkcd'
            elif '截止日期' in col:
                column_mapping[col] = 'Accper'
            elif '报表类型编码' in col:
                column_mapping[col] = 'Typrep'
            elif '行业代码' in col or col == 'Indcd':
                column_mapping[col] = 'Indcd'
            elif '流动比率' in col:
                column_mapping[col] = 'F010101A'
            elif '速动比率' in col:
                column_mapping[col] = 'F010201A'
            elif '利息保障倍数B' in col:
                column_mapping[col] = 'F010702B'
            elif '现金流量' in col and '流动负债' in col:
                column_mapping[col] = 'F010801B'
            elif '资产负债率' in col:
                column_mapping[col] = 'F011201A'
        df = df.rename(columns=column_mapping)
    
    # 标准化键字段
    if 'Stkcd' in df.columns:
        df['Stkcd_std'] = df['Stkcd'].apply(standardize_stock_code)
    if 'Accper' in df.columns:
        df['Year'] = df['Accper'].apply(extract_year)
    
    # 清洗：移除无效记录
    before = len(df)
    df = df[df['Stkcd_std'].notna() & df['Year'].notna()].copy()
    after = len(df)
    if before > after:
        print(f"  清洗无效键: -{before - after} 行")
    
    # 关键优化：按 Typrep 优先级，每个 (Stkcd, Year, Typrep) 只保留一条
    if 'Typrep' in df.columns:
        df = df[df['Typrep'].notna()].copy()
        df['typrep_priority'] = df['Typrep'].map(typrep_priority).fillna(99)
        
        # 按优先级排序后去重
        df = df.sort_values(['Stkcd_std', 'Year', 'Typrep', 'typrep_priority'])
        df = df.drop_duplicates(subset=['Stkcd_std', 'Year', 'Typrep'], keep='first')
        df = df.drop(columns=['typrep_priority'])
        
        print(f"  去重后: {len(df)} 行, 报表类型: {df['Typrep'].value_counts().to_dict()}")
    else:
        # 无 Typrep 的表（如披露财务），按 (Stkcd, Year) 去重
        df = df.drop_duplicates(subset=['Stkcd_std', 'Year'], keep='first')
        print(f"  去重后: {len(df)} 行")
    
    return df

# 测试：读取一个表
print("=" * 80)
print("测试：读取经营能力表")
print("=" * 80)
test_df = load_and_prepare_table('operation', data_files['operation'])
if test_df is not None:
    print(f"\n✓ 成功读取并规范化")
    print(f"数据形状: {test_df.shape}")
    print(f"列名: {list(test_df.columns)[:10]}...")  # 显示前10列


In [ ]:
print("=" * 80)
print("Step 4: 数据集成 - 横向并表（三键策略）")
print("=" * 80)

# 加载所有主题表
dfs = {}
for name, filepath in data_files.items():
    if name != 'violation':
        df = load_and_prepare_table(name, filepath)
        if df is not None:
            dfs[name] = df

print(f"\n成功加载 {len(dfs)} 个财务表")

# 确定主键集合：从经营能力表开始
if 'operation' not in dfs:
    print("❌ 缺少经营能力表！")
else:
    base_df = dfs['operation'].copy()
    merge_keys = ['Stkcd_std', 'Year', 'Typrep']
    
    # 选择基准表的指标列和其他重要字段
    indicator_cols = [c for c in base_df.columns if c.startswith('F')]
    other_cols = [c for c in base_df.columns if c in ['Indcd']]
    keep_cols = merge_keys + other_cols + indicator_cols
    keep_cols = [c for c in keep_cols if c in base_df.columns]
    base_df = base_df[keep_cols].copy()
    
    print(f"\n基准表（经营能力）: {len(base_df)} 行")
    
    # 依次合并其他表（使用 outer join 保留所有数据）
    merge_order = ['profit', 'growth', 'solvency', 'risk', 'pershare', 'dividend']
    
    for name in merge_order:
        if name not in dfs:
            print(f"⚠️ {name} 表不存在，跳过")
            continue
        
        df = dfs[name].copy()
        
        # 选择指标列和其他重要字段
        indicator_cols = [c for c in df.columns if c.startswith('F')]
        other_cols = [c for c in df.columns if c in ['Indcd']]
        
        # 检查是否有 Typrep
        has_typrep = 'Typrep' in df.columns
        
        if has_typrep:
            keep_cols_df = merge_keys + other_cols + indicator_cols
            keys_to_use = merge_keys
        else:
            # 无 Typrep 的表，只使用 (Stkcd, Year)
            keep_cols_df = ['Stkcd_std', 'Year'] + other_cols + indicator_cols
            keys_to_use = ['Stkcd_std', 'Year']
        
        keep_cols_df = [c for c in keep_cols_df if c in df.columns]
        df_subset = df[keep_cols_df].copy()
        
        print(f"\n合并 {name} 表...")
        print(f"  合并前: {len(base_df)} 行")
        print(f"  待合并: {len(df_subset)} 行 (使用键: {keys_to_use})")
        
        # 使用 outer join 保留所有记录
        base_df = base_df.merge(df_subset, on=keys_to_use, how='outer', suffixes=('', '_dup'))
        
        # 删除重复列（如果有）
        dup_cols = [c for c in base_df.columns if c.endswith('_dup')]
        if dup_cols:
            base_df = base_df.drop(columns=dup_cols)
        
        print(f"  合并后: {len(base_df)} 行")
    
    print(f"\n✓ 最终集成数据: {len(base_df)} 行, {len(base_df.columns)} 列")
    merged_df = base_df


In [ ]:
print("=" * 80)
print("Step 5: 生成违规标签")
print("=" * 80)

filepath = data_files['violation']
if not os.path.exists(filepath):
    print("⚠️ 违规信息表不存在")
    merged_df['isviolation'] = 0
else:
    violation_df = read_excel_safe(filepath)
    if violation_df is None:
        merged_df['isviolation'] = 0
    else:
        # 标准化违规表的键
        violation_df['Stkcd_std'] = violation_df['Symbol'].apply(standardize_stock_code)
        violation_df['Year'] = pd.to_numeric(violation_df['ViolationYear'], errors='coerce')
        violation_df = violation_df[violation_df['Year'].notna()].copy()
        violation_df['Year'] = violation_df['Year'].astype(int)
        
        # 确保主表的 Year 是 int
        merged_df['Year'] = merged_df['Year'].astype(int)
        
        # 筛选实际违规记录（IsViolated='Y'）
        violation_yes = violation_df[violation_df['IsViolated'] == 'Y'].copy()
        print(f"实际违规记录数: {len(violation_yes)}")
        
        # 按 (Stkcd_std, Year) 聚合（违规是针对公司年度，不分报表类型）
        violation_set = violation_yes.groupby(['Stkcd_std', 'Year']).size().reset_index(name='violation_count')
        violation_set['isviolation'] = 1
        
        print(f"违规的 (公司, 年份) 数: {len(violation_set)}")
        
        # 合并到主表
        merged_df = merged_df.merge(
            violation_set[['Stkcd_std', 'Year', 'isviolation']], 
            on=['Stkcd_std', 'Year'], 
            how='left'
        )
        merged_df['isviolation'] = merged_df['isviolation'].fillna(0).astype(int)
        
        print(f"✓ 违规样本: {merged_df['isviolation'].sum()} ({merged_df['isviolation'].mean():.2%})")

# 显示违规样本统计
print("\n违规样本统计:")
print(merged_df['isviolation'].value_counts())


## Step 6: 数据清洗

处理缺失值、异常值，确保数据类型正确。


In [ ]:
print("=" * 80)
print("Step 6: 数据清洗")
print("=" * 80)

# 统一缺失值
cleaned_df = merged_df.replace([np.inf, -np.inf], np.nan)

# 统计缺失值
missing_pct = (cleaned_df.isnull().sum() / len(cleaned_df) * 100).round(2)
high_missing = missing_pct[missing_pct > 10].sort_values(ascending=False)

print(f"\n缺失值统计（缺失率>10%的列）:")
for col, pct in high_missing.head(10).items():
    print(f"  {col}: {pct}%")

# 确保指标列为数值类型
numeric_cols = [c for c in cleaned_df.columns if c.startswith('F')]
for col in numeric_cols:
    if col in cleaned_df.columns:
        cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce')

print(f"\n✓ 已确保 {len(numeric_cols)} 个指标列为数值类型")
print(f"数据形状: {cleaned_df.shape}")


## Step 7: 生成ST标签

基于财务异常指标生成 `isST` 标签（ST警示标记）。


In [ ]:
print("\n生成ST标签...")

# 初始化isST为0
cleaned_df['isST'] = 0

# 规则1: 资不抵债（资产负债率>100%，即净资产为负）
if 'F011201A' in cleaned_df.columns:
    condition_insolvency = cleaned_df['F011201A'] > 1.0  # 资产负债率>100%
    cleaned_df.loc[condition_insolvency, 'isST'] = 1
    print(f"  规则1（资不抵债）: {condition_insolvency.sum()} 条")

# 规则2: 净利润为负（ROA为负）
if 'F050204C' in cleaned_df.columns:
    condition_negative_profit = cleaned_df['F050204C'] < 0  # ROA<0表示净利润为负
else:
    condition_negative_profit = False

# 规则3: 有违规记录
condition_violation = cleaned_df['isviolation'] == 1

# 综合判断：满足以下任一条件即标记为ST
# 1. 资不抵债（净资产为负）
# 2. 净利润为负 且 有违规记录
cleaned_df.loc[condition_negative_profit & condition_violation, 'isST'] = 1
print(f"  规则2（净利润为负+违规）: {(condition_negative_profit & condition_violation).sum()} 条")

# 特别处理：连续违规的公司更可能被ST
# 按公司分组，计算累计违规次数
if 'Stkcd_std' in cleaned_df.columns and 'Year' in cleaned_df.columns:
    cleaned_df = cleaned_df.sort_values(['Stkcd_std', 'Year'])
    cleaned_df['cumulative_violations'] = cleaned_df.groupby('Stkcd_std')['isviolation'].cumsum()
    # 累计违规>=2次的标记为ST
    condition_cumulative = cleaned_df['cumulative_violations'] >= 2
    cleaned_df.loc[condition_cumulative, 'isST'] = 1
    print(f"  规则3（累计违规>=2次）: {condition_cumulative.sum()} 条")
    cleaned_df = cleaned_df.drop(columns=['cumulative_violations'])

st_count = cleaned_df['isST'].sum()
st_rate = cleaned_df['isST'].mean()
print(f"\n✓ ST样本: {st_count} ({st_rate:.2%})")
print(f"ST判断依据:")
print(f"  - 资不抵债（资产负债率>100%）")
print(f"  - 净利润为负 + 有违规记录")
print(f"  - 累计违规>=2次")


douxiej

In [ ]:
print("=" * 80)
print("Step 8: 准备最终输出")
print("=" * 80)

# 恢复原始列名
cleaned_df['Stkcd'] = cleaned_df['Stkcd_std'].apply(lambda x: int(x) if pd.notna(x) else np.nan)
cleaned_df['Accper'] = cleaned_df['Year']

# 确保 Typrep 存在
if 'Typrep' not in cleaned_df.columns:
    cleaned_df['Typrep'] = ''

# 确保 Indcd 存在
if 'Indcd' not in cleaned_df.columns:
    cleaned_df['Indcd'] = ''

# 确保所有必需列存在
for col in column_order:
    if col not in cleaned_df.columns:
        if col in ['isviolation', 'isST']:
            cleaned_df[col] = 0
        elif col not in ['Stkcd', 'Accper', 'Typrep', 'Indcd']:
            cleaned_df[col] = np.nan

# 选择并排序列
output_df = cleaned_df[column_order].copy()

# 删除完全重复的行
before = len(output_df)
output_df = output_df.drop_duplicates()
after = len(output_df)
if before > after:
    print(f"删除重复行: -{before - after} 行")

# 排序
output_df = output_df.sort_values(['Stkcd', 'Accper', 'Typrep']).reset_index(drop=True)

# 统计信息
print(f"\n最终数据形状: {output_df.shape}")
print(f"样本量: {len(output_df)}")
print(f"公司数: {output_df['Stkcd'].nunique()}")
print(f"年份范围: {output_df['Accper'].min()}-{output_df['Accper'].max()}")
print(f"报表类型: {output_df['Typrep'].value_counts().to_dict()}")
print(f"违规比例: {output_df['isviolation'].mean():.2%}")
print(f"ST比例: {output_df['isST'].mean():.2%}")

# 显示前几行
print("\n前5行预览:")
print(output_df.head())


doux

In [ ]:
print("=" * 80)
print("Step 9: 质量校验")
print("=" * 80)

print("\n✓ 完整性检查:")
key_completeness = ((output_df['Stkcd'].notna()) & (output_df['Accper'].notna())).mean()
print(f"  主键完整率: {key_completeness:.2%}")

print("\n✓ 一致性检查:")
print(f"  数据量: {len(output_df)} 条")
print(f"  公司数: {output_df['Stkcd'].nunique()} 家")
print(f"  年份: {output_df['Accper'].min()}-{output_df['Accper'].max()}")

print("\n✓ 合理性检查:")
violation_rate = output_df['isviolation'].mean()
print(f"  违规比例: {violation_rate:.2%} (合理范围: 3-10%)")
st_rate = output_df['isST'].mean()
print(f"  ST比例: {st_rate:.2%}")

print("\n✓ 抽样验证:")
sample = output_df.head(3)
for idx, row in sample.iterrows():
    print(f"  Stkcd={int(row['Stkcd'])}, Year={int(row['Accper'])}, Typrep={row['Typrep']}, "
          f"Indcd={row['Indcd']}, isviolation={row['isviolation']}, isST={row['isST']}")

print("\n✓ 质量校验通过")


xie

In [ ]:
print("=" * 80)
print("Step 10: 导出文件")
print("=" * 80)

output_file = os.path.join(OUTPUT_DIR, f'{GROUP_ID}-preprocessed.csv')

try:
    output_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    size_mb = os.path.getsize(output_file) / 1024 / 1024
    print(f"\n✓ 成功保存: {output_file}")
    print(f"✓ 文件大小: {size_mb:.2f} MB")
    print(f"✓ 数据形状: {output_df.shape}")
    
    print(f"\n前5行预览:")
    print(output_df.head())
    
    end_time = datetime.now()
    print(f"\n" + "=" * 80)
    print("数据预处理完成！")
    print("=" * 80)
    print(f"完成时间: {end_time}")
    print(f"输出文件: {output_file}")
    
except Exception as e:
    print(f"❌ 保存失败: {str(e)}")
    import traceback
    traceback.print_exc()


## 📊 数据统计摘要

查看最终数据的详细统计信息。


In [ ]:
# 数据统计摘要
print("=" * 80)
print("数据统计摘要")
print("=" * 80)

print(f"\n1. 基本统计:")
print(f"   总样本数: {len(output_df):,}")
print(f"   公司数: {output_df['Stkcd'].nunique():,}")
print(f"   年份范围: {int(output_df['Accper'].min())}-{int(output_df['Accper'].max())}")
print(f"   特征数: {len(output_df.columns)}")

print(f"\n2. 标签分布:")
print(f"   违规样本: {output_df['isviolation'].sum():,} ({output_df['isviolation'].mean():.2%})")
print(f"   ST样本: {output_df['isST'].sum():,} ({output_df['isST'].mean():.2%})")
print(f"   ST且违规: {((output_df['isST']==1) & (output_df['isviolation']==1)).sum():,} "
      f"({((output_df['isST']==1) & (output_df['isviolation']==1)).mean():.2%})")

print(f"\n3. 报表类型分布:")
print(output_df['Typrep'].value_counts())

print(f"\n4. 行业代码统计:")
if output_df['Indcd'].notna().sum() > 0:
    print(f"   非空行业代码: {output_df['Indcd'].notna().sum():,} ({output_df['Indcd'].notna().mean():.2%})")
    print(f"   唯一行业数: {output_df['Indcd'].nunique()}")
    print(f"   前10个行业:")
    print(output_df['Indcd'].value_counts().head(10))
else:
    print("   所有行业代码为空")

print(f"\n5. 缺失值统计（前10个缺失率最高的列）:")
missing_stats = (output_df.isnull().sum() / len(output_df) * 100).sort_values(ascending=False)
print(missing_stats.head(10))
